In [1]:
import pandas as pd

In [62]:

train_neg = pd.read_csv("train_Arabic_tweets_negative_20190413.tsv", sep="\t").to_numpy()
train_pos = pd.read_csv("train_Arabic_tweets_positive_20190413.tsv", sep="\t").to_numpy()

In [63]:
train_neg.shape, train_pos.shape

((22513, 2), (22760, 2))

In [64]:
train_combined = pd.concat([pd.DataFrame(train_neg), pd.DataFrame(train_pos)], axis=0)
# first column is the label, let's and its neg and pos to 0 and 1 respectively
train_combined[0] = train_combined[0].replace({"neg": 0, "pos": 1})
#store it again 
train_combined.to_csv("train_combined.tsv", sep="\t", index=False, header=False)

C:\Users\sauda\AppData\Local\Temp\ipykernel_6512\3926780181.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_combined[0] = train_combined[0].replace({"neg": 0, "pos": 1})


In [73]:
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def read_file(file_path):
    """Reads a file and returns (text, label)."""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read().strip()

def process_dataset():
    trainset = []
    file_list = []
    
    # Collect all files and labels first
    for case in ["pos", "neg"]:
        path = os.path.join(os.getcwd(), case)
        label = 1 if case == "pos" else 0

        for txt in os.listdir(path):  # No sorting needed for parallel processing
            file_list.append((os.path.join(path, txt), label))

    # Read files in parallel
    with ThreadPoolExecutor(max_workers=8) as executor:  # Adjust based on CPU cores
        texts = list(executor.map(lambda f: read_file(f[0]), file_list))

    # Combine results efficiently
    trainset = np.array([(texts[i], file_list[i][1]) for i in range(len(file_list))], dtype=object)

    return trainset

# Process the dataset
trainset = process_dataset()
print(f"\nDataset processed: {trainset.shape}")  # Verify shape



Dataset processed: (58751, 2)


In [111]:
trainset.shape
# Save the processed dataset to csv
trainset_df = pd.DataFrame(trainset, columns=["text", "label"])
trainset_df.to_csv("trainset.csv", index=False, header=False)

In [7]:
trainset_df = pd.read_csv("trainset.csv", header=None)

In [8]:
trainset_df.shape

(58751, 2)

In [6]:
(trainset_df.iloc[:, 1].value_counts()
)# simple issue, the label in trainset is in column 1 not zeri, while in additional data it is in column 0
# let's fix that
columns_titles = ["label", "text"]
trainset_df = trainset_df.reindex(columns=columns_titles)
trainset_df.iloc[:, 0].value_counts()

Series([], Name: count, dtype: int64)

In [9]:
additional_data = pd.read_csv("ar_reviews_100k.tsv", sep="\t", header=0)
additional_data.iloc[:, 0] = additional_data.iloc[:, 0].replace({"Positive": 1, "Negative": 0, "Mixed": 2})


C:\Users\sauda\AppData\Local\Temp\ipykernel_1432\870861308.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  additional_data.iloc[:, 0] = additional_data.iloc[:, 0].replace({"Positive": 1, "Negative": 0, "Mixed": 2})


In [114]:
print(additional_data.columns) # check the counts of each label
print(additional_data.iloc[:, 0].value_counts()) # check the counts of each label
print(trainset_df.columns) # check the counts of each label
print(trainset_df.iloc[:, 0].value_counts()) # check the counts of each label again

Index(['label', 'text'], dtype='object')
label
1    33333
2    33333
0    33333
Name: count, dtype: int64
Index(['label', 'text'], dtype='object')
label
1    29849
0    28902
Name: count, dtype: int64


In [116]:
# Rename `additional_data` columns to match `trainset_df` if necessary
if 0 in additional_data.columns:  # If columns are numerical indexes
    additional_data.columns = ['label', 'text']

# Convert `label` to int (if needed) for consistency
additional_data['label'] = additional_data['label'].astype(int)
trainset_df['label'] = trainset_df['label'].astype(int)

# Perform union (concatenation)
combined_df = pd.concat([trainset_df, additional_data], ignore_index=True)

# Verify result
print(combined_df.columns)  # Should be ['label', 'text']
print(combined_df['label'].value_counts())  # Check label distribution
print(combined_df.shape)  # Check final dataset size
combined_df.to_csv("combined_dataset.csv", index=False, header=False)  # Save the combined dataset

Index(['label', 'text'], dtype='object')
label
1    63182
0    62235
2    33333
Name: count, dtype: int64
(158750, 2)


In [2]:
combined_df = pd.read_csv("combined_dataset.csv", header=None)
combined_df.count()
combined_df.head()
combined_df.iloc[:4, 1].tolist()

['نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لله، لا تبحثوا فينا عن قوة، إننا مكسورون، القوة التي…',
 'وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال في روحك أماالمنبهرون بالمظا…',
 'نمش ننوم ما دا ديل ولادنا 💚',
 'تعدل النت وشفتها ✌']

In [3]:
from minbpe import RegexTokenizer

In [4]:
trainSet = " ".join(combined_df.iloc[:55000:10, 1].tolist() + combined_df.iloc[60000::10, 1].tolist())

In [5]:
len(trainSet)

3306086

In [6]:
enc = RegexTokenizer()

In [7]:
enc.train(trainSet, 5000, verbose=True)

merge 1/4744: (216, 167) -> 256 (b'\xd8\xa7') had 393242 occurrences
merge 2/4744: (217, 132) -> 257 (b'\xd9\x84') had 282346 occurrences
merge 3/4744: (32, 217) -> 258 (b' \xd9') had 211184 occurrences
merge 4/4744: (32, 216) -> 259 (b' \xd8') had 197586 occurrences
merge 5/4744: (217, 138) -> 260 (b'\xd9\x8a') had 167523 occurrences
merge 6/4744: (256, 257) -> 261 (b'\xd8\xa7\xd9\x84') had 140694 occurrences
merge 7/4744: (217, 134) -> 262 (b'\xd9\x86') had 126243 occurrences
merge 8/4744: (32, 261) -> 263 (b' \xd8\xa7\xd9\x84') had 110499 occurrences
merge 9/4744: (256, 216) -> 264 (b'\xd8\xa7\xd8') had 109106 occurrences
merge 10/4744: (217, 133) -> 265 (b'\xd9\x85') had 102919 occurrences
merge 11/4744: (216, 177) -> 266 (b'\xd8\xb1') had 92319 occurrences
merge 12/4744: (217, 136) -> 267 (b'\xd9\x88') had 84497 occurrences
merge 13/4744: (217, 135) -> 268 (b'\xd9\x87') had 82004 occurrences
merge 14/4744: (216, 170) -> 269 (b'\xd8\xaa') had 80580 occurrences
merge 15/4744: (216, 

KeyboardInterrupt: 

In [8]:
enc.save(f"sen_twitter3-{len(enc.vocab)}")

In [10]:
enc.merges

{}

In [42]:
enc = RegexTokenizer()
enc.train(trainSet, 1000, verbose=True)
enc.save("sen_twitter2-1000")

merge 1/744: (216, 167) -> 256 (b'\xd8\xa7') had 341930 occurrences
merge 2/744: (217, 132) -> 257 (b'\xd9\x84') had 261572 occurrences
merge 3/744: (217, 138) -> 258 (b'\xd9\x8a') had 186425 occurrences
merge 4/744: (32, 216) -> 259 (b' \xd8') had 181986 occurrences
merge 5/744: (32, 217) -> 260 (b' \xd9') had 154965 occurrences
merge 6/744: (256, 257) -> 261 (b'\xd8\xa7\xd9\x84') had 118995 occurrences
merge 7/744: (217, 134) -> 262 (b'\xd9\x86') had 114787 occurrences
merge 8/744: (256, 216) -> 263 (b'\xd8\xa7\xd8') had 95713 occurrences
merge 9/744: (217, 133) -> 264 (b'\xd9\x85') had 92545 occurrences
merge 10/744: (217, 136) -> 265 (b'\xd9\x88') had 85490 occurrences
merge 11/744: (32, 261) -> 266 (b' \xd8\xa7\xd9\x84') had 82022 occurrences
merge 12/744: (217, 135) -> 267 (b'\xd9\x87') had 81638 occurrences
merge 13/744: (216, 177) -> 268 (b'\xd8\xb1') had 80233 occurrences
merge 14/744: (216, 170) -> 269 (b'\xd8\xaa') had 64754 occurrences
merge 15/744: (216, 168) -> 270 (b'\xd

In [2]:
ord("0")

48